In [ ]:
#!pip install PyYAML

In [2]:
#pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl (38.6 MB)
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [19]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [20]:
# Load YAML
with open('data.yaml', mode='r') as f:
    data_yaml = yaml.load(f,Loader=SafeLoader)


labels = data_yaml['names']
print(labels)

['IssueBank', 'ReceiverName', 'AccountNo', 'ChequeNo.', 'DateIssue', 'Ammount', 'Sign']


In [21]:
# Load YOLO Model
yolo = cv2.dnn.readNetFromONNX('./Model/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [22]:
# Load the image
img = cv2.imread('chq.jpg.jpg')
image = img.copy()
row, col, d = image.shape

# Get the YOLO prediction from the image
# step-1 convert image into square image (array)
max_rc = max(row, col)
input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
input_image[0:row,0:col] = image

# step-2: get prediction from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop=False)
yolo.setInput(blob)
preds = yolo.forward()   # detection or prediction from YOLO

In [23]:
print(preds)

[[[4.0168505e+00 1.7234898e+00 2.7567961e+01 ... 1.8539418e-01
   6.3932128e-02 6.9228768e-02]
  [1.2838647e+01 1.4932866e+00 2.8103695e+01 ... 1.7429283e-01
   3.9177015e-02 7.1260631e-02]
  [2.0332840e+01 1.2429647e+00 3.1333464e+01 ... 1.7109732e-01
   3.3423785e-02 7.0292294e-02]
  ...
  [5.7076135e+02 6.3433179e+02 5.0670584e+02 ... 1.3442059e-01
   9.5744498e-02 1.9229336e-01]
  [6.0134656e+02 6.3375299e+02 5.2751019e+02 ... 1.4253397e-01
   9.2595249e-02 1.9089629e-01]
  [6.3580927e+02 6.3139709e+02 4.4287405e+02 ... 1.5619877e-01
   9.1134556e-02 1.7358252e-01]]]


In [24]:
print(preds.shape)

(1, 25200, 12)


In [25]:
# Non Maximum Supression
# step-1: filter detection based on confidence (0.4) and probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

# width and height of the image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] # confidence of detection on object
    if confidence > 0.4:
        class_score = row[5:].max() # maximum probability from 7 objects
        class_id = row[5:].argmax() # get the index position at which max probability occur

        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            # construct bounding from four values
            # Left, top, width, height
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)

            box = np.array([left, top, width, height])

            # append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)


# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.45).flatten()

In [26]:
index

array([23, 39, 35, 40, 27, 32, 52])

In [11]:
# Draw the Bounding
for ind in index:
    # extract bounding box
    x, y, w, h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]

    text = f'{class_name}: {bb_conf}%'

    cv2.rectangle(image,(x,y),(x+w, y+h), (0, 255, 0), 2)
    cv2.rectangle(image,(x,y-30),(x+w, y), (255, 255, 255), -1)

    cv2.putText(image,text,(x,y-10), cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)    

In [12]:
# cv2.imshow('original',img)
# cv2.imshow('yolo_prediction',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [27]:
# Draw the Bounding
for ind in index:
    # extract bounding box
    x, y, w, h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]

    text = f'{class_name}: {bb_conf}%'

    cv2.rectangle(image,(x,y),(x+w, y+h), (0, 255, 0), 2)
    cv2.rectangle(image,(x,y-30),(x+w, y), (255, 255, 255), -1)

    # Increase font size by changing fontScale parameter
    cv2.putText(image, text, (x,y-10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0,0,0), 2)


In [28]:
# Resize the output image to a smaller dimension for zooming out
zoom_out_scale = 0.6  # You can adjust this value to control the zoom level

# Calculate the new dimensions for zooming out
new_width = int(image.shape[1] * zoom_out_scale)
new_height = int(image.shape[0] * zoom_out_scale)

# Resize the image to the new dimensions
zoomed_out_image = cv2.resize(image, (new_width, new_height))

# Display the zoomed out image
cv2.imshow('Zoomed Out Image', zoomed_out_image)
cv2.waitKey(0)
cv2.destroyAllWindows()